## Summary

* One of the issues I noticed with the previous regression model when I was testing on QPO frequencies was although it could predict Frequencies for true QPOs well, the network was very bad at predicting 0 as the frequency for "qpos" that don't exist, i.e. it would predict 0.6 for the frequency of a third qpo when that pds only had two qpos. See this plot: 

![](https://raw.githubusercontent.com/personal-research/MAXI-J1535/main/archive/images/frequency_regression.png "Initial Frequency Regression")

* My proposed solution is this: combine the classification model I designed in ```interim.ipynb``` with the regression model I designed in ```exploring_regression_architectures.ipynb```. Since the classification model could predict (with ~85% accuracy) the number of QPOs, what if I make the model predict the number of QPOs first, and then automatically return 0s for outputs of the "buffer" qpos. For example, if the model predicts that there is only one qpo, it will only run regression for one frequency and return the frequency array ```[freq, 0, 0]``` rather than attempting to fit something weird for those second two frequencies when they should be zero. 

## Notes on Models for Combined Classification and Regression

* [machine learning mastery article](https://machinelearningmastery.com/neural-network-models-for-combined-classification-and-regression/)

```python
# split into input (X) and output (y) variables
X, y = dataset[:, 1:-1], dataset[:, -1]
X, y = X.astype('float'), y.astype('float')
n_features = X.shape[1]
# encode strings to integer
y_class = LabelEncoder().fit_transform(y)
n_class = len(unique(y_class))
# split data into train and test sets
X_train, X_test, y_train, y_test, y_train_class, y_test_class = train_test_split(X, y, y_class, test_size=0.33, random_state=1)
# input
visible = Input(shape=(n_features,))
hidden1 = Dense(20, activation='relu', kernel_initializer='he_normal')(visible)
hidden2 = Dense(10, activation='relu', kernel_initializer='he_normal')(hidden1)
# regression output
out_reg = Dense(1, activation='linear')(hidden2)
# classification output
out_clas = Dense(n_class, activation='softmax')(hidden2)
# define model
model = Model(inputs=visible, outputs=[out_reg, out_clas])
# compile the keras model
model.compile(loss=['mse','sparse_categorical_crossentropy'], optimizer='adam')
```

![](https://machinelearningmastery.com/wp-content/uploads/2021/02/Plot-of-the-Multi-Output-Model-for-Combine-Regression-and-Classification-Predictions.jpg "Network Flowchart")

## Model Merging in Keras
* [stack exchange article](https://datascience.stackexchange.com/questions/26103/merging-two-different-models-in-keras)

![](https://i.stack.imgur.com/2xIdb.png "Network Flowchart")


```python
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model
A1 = Input(shape=(30,),name='A1')
A2 = Dense(8, activation='relu',name='A2')(A1)
A3 = Dense(30, activation='relu',name='A3')(A2)

B2 = Dense(40, activation='relu',name='B2')(A2)
B3 = Dense(30, activation='relu',name='B3')(B2)

merged = Model(inputs=[A1],outputs=[A3,B3])
```

## Update
* I think I'll either use a custom layer or custom activation function. Currently pursuing the latter. 

## Code

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from keras import models, layers
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model

np.set_printoptions(suppress=True)
data_df = pd.read_csv('https://github.com/personal-research/MAXI-J1535/raw/main/data/processed/full.csv')

KeyboardInterrupt: 

In [ ]:
data_df = data_df.sample(frac=1) # shuffle the dataframe

def normalize(x):
    x = (x-min(x))/(max(x)-min(x))
    return x

# X

arr_names =  ['hardness', 'tins', 'disk_norm', 'gammas', 'nthcomp_norms', 'intensities']
(hardness, tins, disk_norms, gammas, nthcomp_norms, intensities) = (normalize(np.array(data_df[arr_name])) for arr_name in arr_names)

## Stack x values
x_vals = np.array([], dtype=np.int64).reshape(0,6) # why are they type int

for a, b, c, d, e, f in zip(hardness, tins, disk_norms, gammas, nthcomp_norms, intensities): 
    new_arr = np.array([float(a), float(b), float(c), float(d), float(e), float(f)])
    x_vals = np.vstack([x_vals, new_arr])

# Y 
y_vals = np.array([], dtype=np.float32).reshape(0, 3)

freq1s, freq2s, freq3s = (normalize(np.array(data_df[arr_name])) for arr_name in ['first_par1s', 'second_par1s', 'third_par1s'])

for freq1, freq2, freq3 in zip(freq1s, freq2s, freq3s): 
    new_arr = np.array([float(freq1), float(freq2), float(freq3)])
    y_vals = np.vstack([y_vals, new_arr])

# Train test split
X_train, X_test, y_train, y_test = train_test_split(x_vals, y_vals, test_size=0.2)

In [ ]:
import numpy as np

In [4]:
# custom function
#from keras import backend as K
def cullBuffers(x, qpo_classes): # qpo_classes needs to be determined before hand
    new_x = np.array([], dtype=np.float64).reshape(0,3)
    for x_row, qpo_class in zip(x, qpo_classes): 
        if qpo_class == 0: 
            new_x = np.vstack([new_x, np.zeros(3)])

        elif qpo_class == 1:
            new_x = np.vstack([new_x, [x_row[0], 0, 0]])

        elif qpo_class == 2: 
            new_x = np.vstack([new_x, [x_row[0], x_row[1], 0]])

        elif qpo_class == 3: 
            new_x = np.vstack([new_x, x_row[0:3]])

    return new_x

In [5]:
print(cullBuffers([[1., 1., 1.], [2., 2., 2.], [3., 3., 3.], [4., 4., 4.,]], [0, 1, 2, 3]))

[[0. 0. 0.]
 [2. 0. 0.]
 [3. 3. 0.]
 [4. 4. 4.]]
